In [1]:
import os
from woocommerce import API
from dotenv import load_dotenv

load_dotenv()

woocommerce_url = os.getenv('URL')
woocommerce_key = os.getenv('KEY')
woocommerce_secret = os.getenv('SECRET')

wc_api = API(
    url=woocommerce_url,
    consumer_key=woocommerce_key,
    consumer_secret=woocommerce_secret,
    version='wc/v3'
)


In [2]:
params = {
    'search': 'SAUCONY',
    'status': 'publish',
    'password_protected': False,
    'on_sale': 'true',
    'per_page': 95
}


In [7]:
all_saucony_products = []

page = 1
while True:
    params['page'] = page
    products = wc_api.get("products", params=params).json()
    if not products:
        break
    all_saucony_products.extend(products)
    page += 1


In [12]:
originals_sku_list = []
originals_list = []
originals_id_list = []

for product in all_saucony_products:
    attributes = product.get('attributes', [])
    for attribute in attributes:
        if attribute.get('name') == 'Brend' and 'Saucony Originals' in attribute.get('options', []):
            originals_id_list.append(product['id'])

In [187]:
len(originals_id_list)

105

In [188]:
for i in originals_id_list:
    product = wc_api.get(f'products/{i}').json()
    print(product['date_created'])

2023-08-01T22:42:14
2023-07-01T22:42:12
2023-03-01T22:42:12
2023-07-01T22:42:10
2023-03-01T22:42:10
2023-03-01T22:42:07
2023-07-01T22:42:07
2023-07-01T22:42:05
2023-07-01T22:42:05
2023-07-01T22:42:03
2023-07-01T22:42:03
2023-07-01T22:41:53
2023-03-01T14:56:30
2023-03-01T14:56:27
2023-03-01T14:56:27
2023-03-01T14:56:25
2023-03-01T14:56:25
2023-03-01T14:56:25
2023-03-01T14:56:24
2023-07-28T14:56:30
2023-07-28T14:56:20
2023-07-28T14:56:20
2023-07-28T14:56:18
2023-07-28T14:56:18
2023-02-28T14:56:16
2023-02-28T14:56:16
2023-02-28T14:56:14
2023-07-28T14:56:14
2023-07-08T15:54:08
2023-07-08T15:54:08
2023-07-08T15:53:54
2023-07-08T15:53:54
2023-07-08T15:53:54
2023-03-23T18:01:14
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00


In [ ]:
wc_api.get("products", params={"per_page": 20}).json()

In [30]:
product_sku = '006272-108'
product = wc_api.get('products', params={"sku":product_sku}).json()

In [16]:
low_originals = []

for product_id in originals_id_list:
    total_stock = 0 
    variations = wc_api.get(f"products/{product_id}/variations").json()

    for variation in variations:
        stock_quantity = variation['stock_quantity']
        total_stock += stock_quantity if stock_quantity is not None else 0
    
    if total_stock <= 3:
        low_originals.append(product_id)


In [22]:
print(originals_id_list)

[10862, 10838, 10829, 10811, 10798, 10790, 10783, 10768, 10759, 10749, 10737, 10636, 18478, 18458, 18447, 18429, 18423, 18418, 18407, 18468, 18369, 18358, 18350, 18335, 18325, 18319, 18309, 18290, 9360, 9351, 9213, 9204, 9193, 14689, 14677, 14673, 14669, 14666, 14661, 14646, 14644, 14627, 14618, 14616, 14614, 14573, 14570, 14542, 14503, 14499, 14495, 14477, 14207, 14205, 14201, 14197, 14189, 14181, 14154, 14122, 14117, 14110, 14102, 14099, 14051, 14049, 14046, 14042, 14034, 14028, 14026, 14022, 14020, 14016, 14013, 14004, 11030, 11025, 11018, 11008, 7832, 7818, 7809, 7796, 7788, 7781, 7769, 7760, 7751, 7742, 7734, 6590, 6582, 6566, 6559, 6551, 6544, 6534, 6525, 6520, 6514, 6506, 10311, 10298, 10848]


In [59]:
print(low_originals)

[14689, 14677, 14673, 14669, 14666, 14646, 14644, 14618, 14616, 14614, 14573, 14570, 14503, 14477, 14207, 14205, 14117, 14051, 14049, 14046, 14042, 14028, 14026, 14016, 14013, 6590, 6566, 6559, 6551, 6520, 10298]


In [52]:
def add_categories_to_product(product_id, new_category_ids):
    product = wc_api.get(f"products/{product_id}").json()
    existing_category_ids = [category['id'] for category in product['categories']]
    
    categories_to_update = list(set(existing_category_ids + new_category_ids))
    
    updated_categories = [{'id': cat_id} for cat_id in categories_to_update]

    updated_product = {'categories': updated_categories}
    print(wc_api.put(f"products/{product_id}", data=updated_product))

In [ ]:
category_ids_to_add = [151, 132, 152]

In [ ]:
for single_original in low_originals:
  add_categories_to_product(single_original, category_ids_to_add)

In [60]:
data = {
    "date_created": "2017-03-23T17:01:14"
}
print(wc_api.put("products/14689", data))

<Response [200]>


In [72]:
sorted_product_ids = sorted(low_originals[:5], key=get_product_price, reverse=True)

In [ ]:
wc_api.get("products/14690").json()

In [84]:
product_id = 14689

product = wc_api.get(f"products/{product_id}").json()

if 'variations' in product and len(product['variations']) > 0:
    variation_id = product['variations'][0]
    variation = wc_api.get(f"products/{variation_id}").json()

    regular_price = float(
        variation['regular_price']) if 'regular_price' in variation else 0.0
    sale_price = float(variation['sale_price']
                       ) if 'sale_price' in variation else 0.0

    if regular_price != 0:
        percentage_difference = (
            (regular_price - sale_price) / regular_price) * 100
        print(
            f"The percentage difference between regular price and sale price is: {percentage_difference:.2f}%")
    else:
        print("Regular price is not available.")
else:
    print("Product has no variations.")


The percentage difference between regular price and sale price is: 50.00%


In [120]:
percentage_ranges = {
    '0-10': [],
    '11-20': [],
    '21-30': [],
    '31-40': [],
    '41-50': [],
    '51-60': [],
    '61-70': [],
    '71-80': [],
}

# Function to calculate the percentage difference


def calculate_percentage_difference(regular_price, sale_price):
    if regular_price != 0:
        percentage_diff = ((regular_price - sale_price) / regular_price) * 100
        rounded_percentage = round(percentage_diff, 1)
        return rounded_percentage
    return 0.0


# Categorize products into percentage difference ranges
for product_id in originals_id_list:
    product = wc_api.get(f"products/{product_id}").json()
    if 'variations' in product and len(product['variations']) > 0:
        variation_id = product['variations'][0]
        variation = wc_api.get(f"products/{variation_id}").json()

        regular_price = float(
            variation['regular_price']) if 'regular_price' in variation else 0.0
        sale_price = float(variation['sale_price']
                           ) if 'sale_price' in variation else 0.0
        
        percentage_difference = calculate_percentage_difference(
            regular_price, sale_price)
        print(percentage_difference)

        for range_name, range_values in percentage_ranges.items():
            start, end = map(int, range_name.split('-'))
            if start <= percentage_difference <= end:
                range_values.append(product_id)
                break

20.0
30.0
40.0
30.0
40.0
40.0
30.0
30.0
30.0
30.0
30.0
30.0
40.0
40.0
40.0
40.0
40.0
50.0
40.0
30.0
30.0
30.0
30.0
30.0
40.0
40.0
40.0
30.0
30.0
30.0
30.0
30.0
30.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
60.0
60.0
60.0
60.0
60.0
50.0
50.0
50.0
50.0
40.0
40.0
50.0
50.0
50.0
30.0
30.0
50.0
50.0
50.0
40.0
50.0
30.0
50.0
50.0
50.0
50.0
50.0
50.0
30.0
30.0
30.0
30.0
30.0
30.0
40.0
50.0
40.0
50.0
40.0
40.0
50.0
50.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
30.0


In [184]:
total_products = sum(len(product_ids)for product_ids in percentage_ranges.values())
total_products

105

In [131]:
print(percentage_ranges)

{'0-10': [], '11-20': [10862], '21-30': [9360, 10768, 10759, 10848, 11025, 11018, 10783, 9351, 18468, 18369, 18358, 10838, 10811, 7832, 18335, 18290, 14034, 14110, 10749, 10737, 10636, 18350, 9213, 9204, 9193, 11030, 11008, 14102, 14004], '31-40': [10311, 10829, 10798, 10790, 14046, 18478, 7769, 18447, 18429, 18423, 18309, 7796, 7742, 7734, 18458, 7818, 7781, 6590, 6559, 18325, 6566, 6544, 6534, 6520, 6514, 10298, 18407, 18319, 14189, 14181, 6582, 6551, 6525, 6506], '41-50': [18418, 14689, 14677, 14673, 14669, 14666, 14661, 14646, 14644, 14627, 14618, 14616, 14614, 14573, 14570, 14207, 14205, 14201, 14197, 14154, 14122, 14117, 14099, 14051, 14049, 14042, 14028, 14026, 14022, 14020, 14016, 14013, 7809, 7788, 7760, 7751], '51-60': [14542, 14503, 14499, 14495, 14477], '61-70': [], '71-80': []}


In [161]:
def get_product_price(product_id):
    product = wc_api.get(f"products/{product_id}").json()
    if product['price'] == '':
        print(product_id)
    return float(product['price']) if product['price'] != '' else 0.0


In [162]:
for range_name, product_ids in percentage_ranges.items():
    sorted_product_ids = sorted(product_ids, key=get_product_price, reverse=True)
    percentage_ranges[range_name] = sorted_product_ids

14646
14026


In [172]:
print(percentage_ranges)

{'0-10': [], '11-20': [10862], '21-30': [9360, 10768, 10759, 10848, 11025, 11018, 10783, 9351, 18468, 18369, 18358, 10838, 10811, 7832, 18335, 18290, 14034, 14110, 10749, 10737, 10636, 18350, 9213, 9204, 9193, 11030, 11008, 14102, 14004], '31-40': [10311, 10829, 10798, 10790, 14046, 18478, 7769, 18447, 18429, 18423, 18309, 7796, 7742, 7734, 18458, 7818, 7781, 6590, 6559, 18325, 6566, 6544, 6534, 6520, 6514, 10298, 18407, 18319, 14189, 14181, 6582, 6551, 6525, 6506], '41-50': [7809, 7788, 14154, 14661, 7760, 18418, 7751, 14197, 14677, 14122, 14117, 14689, 14666, 14207, 14205, 14201, 14042, 14051, 14669, 14614, 14570, 14099, 14673, 14644, 14618, 14616, 14573, 14049, 14028, 14022, 14020, 14016, 14013, 14627, 14646, 14026], '51-60': [14499, 14495, 14503, 14542, 14477], '61-70': [], '71-80': []}


In [189]:
from datetime import datetime, timedelta

In [201]:
product_ids = [9360, 10768, 10759, 10848, 11025, 11018, 10783, 9351, 18468, 18369, 18358, 10838, 10811,
               7832, 18335, 18290, 14034, 14110, 10749, 10737, 10636, 18350, 9213, 9204, 9193, 11030, 11008, 14102, 14004]
starting_time = datetime(2023, 4, 2, 10, 3, 2)

In [202]:
for product_id in reversed(product_ids):
    new_time_str = starting_time.strftime('%Y-%m-%dT%H:%M:%S')

    updated_product = {'date_created': new_time_str}
    response = wc_api.put(f"products/{product_id}", data=updated_product)

    if response.status_code == 200:
        print(f"Updated date_created for product {product_id}")
    else:
        print(f"Failed to update date_created for product {product_id}")
    
    starting_time += timedelta(minutes=10)


Updated date_created for product 14004
Updated date_created for product 14102
Updated date_created for product 11008
Updated date_created for product 11030
Updated date_created for product 9193
Updated date_created for product 9204
Updated date_created for product 9213
Updated date_created for product 18350
Updated date_created for product 10636
Updated date_created for product 10737
Updated date_created for product 10749
Updated date_created for product 14110
Updated date_created for product 14034
Updated date_created for product 18290
Updated date_created for product 18335
Updated date_created for product 7832
Updated date_created for product 10811
Updated date_created for product 10838
Updated date_created for product 18358
Updated date_created for product 18369
Updated date_created for product 18468
Updated date_created for product 9351
Updated date_created for product 10783
Updated date_created for product 11018
Updated date_created for product 11025
Updated date_created for produ

In [183]:
arr = [7809, 7788, 14154, 14661, 7760, 18418, 7751, 14197, 14677, 14122, 14117, 14689, 14666, 14207, 14205, 14201, 14042, 14051,
       14669, 14614, 14570, 14099, 14673, 14644, 14618, 14616, 14573, 14049, 14028, 14022, 14020, 14016, 14013, 14627, 14646, 14026]

for i in arr[:2]:
    product = wc_api.get(f'products/{i}').json()
    print(type(product['price']))

<class 'str'>
<class 'str'>


In [177]:
product = wc_api.get("products/14026").json()
product

{'id': 14026,
 'name': 'SAUCONY Shadow 5000 Vintage',
 'slug': 'saucony-shadow-5000-vintage-2',
 'permalink': 'https://www.jump2sport.hr/p/saucony-shadow-5000-vintage-2/',
 'date_created': '2023-02-06T00:00:00',
 'date_created_gmt': '2023-02-05T23:00:00',
 'date_modified': '2023-08-30T18:17:27',
 'date_modified_gmt': '2023-08-30T16:17:27',
 'type': 'variable',
 'status': 'publish',
 'featured': False,
 'catalog_visibility': 'visible',
 'description': '',
 'short_description': '<p>Ženske tenisice</p>\n',
 'sku': 'S60405-17',
 'price': '',
 'regular_price': '',
 'sale_price': '',
 'date_on_sale_from': None,
 'date_on_sale_from_gmt': None,
 'date_on_sale_to': None,
 'date_on_sale_to_gmt': None,
 'on_sale': True,
 'purchasable': False,
 'total_sales': 1,
 'virtual': False,
 'downloadable': False,
 'downloads': [],
 'download_limit': -1,
 'download_expiry': -1,
 'external_url': '',
 'button_text': '',
 'tax_status': 'none',
 'tax_class': '',
 'manage_stock': True,
 'stock_quantity': 1,
 'ba